In [2]:
%run _init_.py

C:\Users\m\4차4차\speed\ml\_init_.py:26: SyntaxWarning: invalid escape sequence '\W'
  font_path = "c:\Windows\Fonts\malgun.ttf"


pandas: pd 2.2.2
numpy : np 2.1.0
matplotlib : mpl 3.9.2
seaborn : sns 0.13.2
sklearn : skl 1.5.1


In [3]:
df = pd.read_csv('df.csv')

In [4]:
import pandas as pd

# 필요한 열 리스트
selected_columns = [
    'OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', 
    '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt', 'YearRemodAdd', 
    'MasVnrArea', 'Fireplaces', 'GarageYrBlt', 'HeatingQC', 'GarageType', 
    'FireplaceQu', 'GarageFinish', 'KitchenQual', 'BsmtQual', 'ExterQual', 
    'SalePrice'
]

# 데이터프레임에서 선택된 열만 추출
top20 = df[selected_columns]

# 새로운 데이터프레임을 CSV 파일로 저장
top20.to_csv('top20.csv', index=False)

print("CSV 파일로 저장 완료!")


CSV 파일로 저장 완료!


### 랜덤 포레스트

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# 데이터 준비
X = top20.drop(columns=['SalePrice'])
y = top20['SalePrice']

# 데이터 분리 (80% 훈련, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 스케일러 적용 (StandardScaler)
scaler = StandardScaler()

# 훈련 데이터 스케일링
X_train_scaled = scaler.fit_transform(X_train)

# 테스트 데이터 스케일링 (훈련 데이터의 스케일을 기준으로)
X_test_scaled = scaler.transform(X_test)

# 모델 학습
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# 예측
y_pred = model.predict(X_test_scaled)

# 성능 평가
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse}")


RMSE: 28034.559413997587


c:\Users\m\miniconda3\envs\ml\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 데이터 프레임 로드 (실제 데이터 파일을 불러올 수 있음)
# df = pd.read_csv('your_data.csv')

# 피처와 타겟 변수 설정
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 그레디언트 부스팅 모델 생성
model = GradientBoostingRegressor(random_state=42)

# 하이퍼파라미터 후보 설정
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

# GridSearchCV를 사용하여 하이퍼파라미터 튜닝
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터와 모델 가져오기
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(f"Best Parameters: {best_params}")

# 튜닝된 모델로 훈련
best_model.fit(X_train, y_train)

# 예측
y_pred = best_model.predict(X_test)

# 모델 평가
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
from sklearn.model_selection import cross_val_score

# 교차 검증을 사용하여 모델 평가
cv_scores = cross_val_score(best_model, X_scaled, y, cv=2, scoring='neg_mean_squared_error')
print(f"Cross-Validated MSE: {-cv_scores.mean()}")

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")


Best Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 150}
Cross-Validated MSE: 877654164.8898258
Mean Squared Error: 570017554.9260039
Mean Absolute Error: 15672.526584878266
R^2 Score: 0.9066536520194749


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, VotingRegressor, StackingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression

# 데이터 프레임 로드 (실제 데이터 파일을 불러올 수 있음)
# df = pd.read_csv('your_data.csv')

# 피처와 타겟 변수 설정
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 개별 모델 생성
gbm = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# 보팅 회귀 모델 생성
voting_regressor = VotingRegressor(estimators=[('gbm', gbm), ('rf', rf)])

# 스태킹 회귀 모델 생성
stacking_regressor = StackingRegressor(
    estimators=[('gbm', gbm), ('rf', rf)],
    final_estimator=LinearRegression()
)

# 모델 훈련
voting_regressor.fit(X_train, y_train)
stacking_regressor.fit(X_train, y_train)

# 예측
voting_pred = voting_regressor.predict(X_test)
stacking_pred = stacking_regressor.predict(X_test)

# 모델 평가
voting_mse = mean_squared_error(y_test, voting_pred)
stacking_mse = mean_squared_error(y_test, stacking_pred)
voting_mae = mean_absolute_error(y_test, voting_pred)
stacking_mae = mean_absolute_error(y_test, stacking_pred)
voting_r2 = r2_score(y_test, voting_pred)
stacking_r2 = r2_score(y_test, stacking_pred)

print(f"Voting Regressor - Mean Squared Error: {voting_mse}")
print(f"Voting Regressor - Mean Absolute Error: {voting_mae}")
print(f"Voting Regressor - R^2 Score: {voting_r2}")

print(f"Stacking Regressor - Mean Squared Error: {stacking_mse}")
print(f"Stacking Regressor - Mean Absolute Error: {stacking_mae}")
print(f"Stacking Regressor - R^2 Score: {stacking_r2}")


Voting Regressor - Mean Squared Error: 582372868.4968954
Voting Regressor - Mean Absolute Error: 15650.977845201774
Voting Regressor - R^2 Score: 0.9046303399477148
Stacking Regressor - Mean Squared Error: 572981867.1831857
Stacking Regressor - Mean Absolute Error: 15861.227355198396
Stacking Regressor - R^2 Score: 0.9061682148235666


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import (GradientBoostingRegressor, RandomForestRegressor, 
                              AdaBoostRegressor, ExtraTreesRegressor, 
                              VotingRegressor, StackingRegressor)
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 데이터 프레임 로드 (실제 데이터 파일을 불러올 수 있음)
# df = pd.read_csv('your_data.csv')

# 피처와 타겟 변수 설정
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 개별 모델 생성
gbm = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
rf = RandomForestRegressor(n_estimators=100, random_state=42)
ada = AdaBoostRegressor(n_estimators=100, random_state=42)
extra_trees = ExtraTreesRegressor(n_estimators=100, random_state=42)
linear = LinearRegression()

# 보팅 회귀 모델 생성
voting_regressor = VotingRegressor(estimators=[('gbm', gbm), ('rf', rf), ('ada', ada), ('extra', extra_trees), ('linear', linear)])

# 스태킹 회귀 모델 생성
stacking_regressor = StackingRegressor(
    estimators=[('gbm', gbm), ('rf', rf), ('ada', ada), ('extra', extra_trees)],
    final_estimator=linear
)

# 모델 훈련
voting_regressor.fit(X_train, y_train)
stacking_regressor.fit(X_train, y_train)

# 예측
voting_pred = voting_regressor.predict(X_test)
stacking_pred = stacking_regressor.predict(X_test)

# 모델 평가
voting_mse = mean_squared_error(y_test, voting_pred)
stacking_mse = mean_squared_error(y_test, stacking_pred)
voting_mae = mean_absolute_error(y_test, voting_pred)
stacking_mae = mean_absolute_error(y_test, stacking_pred)
voting_r2 = r2_score(y_test, voting_pred)
stacking_r2 = r2_score(y_test, stacking_pred)

print(f"Voting Regressor - Mean Squared Error: {voting_mse}")
print(f"Voting Regressor - Mean Absolute Error: {voting_mae}")
print(f"Voting Regressor - R^2 Score: {voting_r2}")

print(f"Stacking Regressor - Mean Squared Error: {stacking_mse}")
print(f"Stacking Regressor - Mean Absolute Error: {stacking_mae}")
print(f"Stacking Regressor - R^2 Score: {stacking_r2}")


Voting Regressor - Mean Squared Error: 635651158.5053998
Voting Regressor - Mean Absolute Error: 16419.259756012758
Voting Regressor - R^2 Score: 0.8959054616418409
Stacking Regressor - Mean Squared Error: 562969591.0289302
Stacking Regressor - Mean Absolute Error: 15780.144362520996
Stacking Regressor - R^2 Score: 0.9078078299650574


In [15]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pandas as pd

# 데이터 프레임 로드 (실제 데이터 파일을 불러올 수 있음)
# df = pd.read_csv('your_data.csv')

# 피처와 타겟 변수 설정
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# GradientBoostingRegressor의 다양한 하이퍼파라미터 설정
# 기본 모델
gbm_default = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# 추가 하이퍼파라미터 모델 1
gbm_additional1 = GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=5, random_state=42)

# 추가 하이퍼파라미터 모델 2
gbm_additional2 = GradientBoostingRegressor(n_estimators=50, learning_rate=0.2, max_depth=2, random_state=42)

# 모델 훈련 및 예측
def train_and_evaluate(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    return mse, mae, r2

# 기본 모델 평가
mse_default, mae_default, r2_default = train_and_evaluate(gbm_default, X_train, X_test, y_train, y_test)
print(f"Default Model - Mean Squared Error: {mse_default}")
print(f"Default Model - Mean Absolute Error: {mae_default}")
print(f"Default Model - R^2 Score: {r2_default}")

# 추가 하이퍼파라미터 모델 1 평가
mse_additional1, mae_additional1, r2_additional1 = train_and_evaluate(gbm_additional1, X_train, X_test, y_train, y_test)
print(f"Additional Model 1 - Mean Squared Error: {mse_additional1}")
print(f"Additional Model 1 - Mean Absolute Error: {mae_additional1}")
print(f"Additional Model 1 - R^2 Score: {r2_additional1}")

# 추가 하이퍼파라미터 모델 2 평가
mse_additional2, mae_additional2, r2_additional2 = train_and_evaluate(gbm_additional2, X_train, X_test, y_train, y_test)
print(f"Additional Model 2 - Mean Squared Error: {mse_additional2}")
print(f"Additional Model 2 - Mean Absolut")


Default Model - Mean Squared Error: 571375777.6040524
Default Model - Mean Absolute Error: 15819.384795998207
Default Model - R^2 Score: 0.9064312288227777
Additional Model 1 - Mean Squared Error: 609409214.555808
Additional Model 1 - Mean Absolute Error: 15814.155187585548
Additional Model 1 - R^2 Score: 0.9002028549597046
Additional Model 2 - Mean Squared Error: 639000864.9621615
Additional Model 2 - Mean Absolut


GradientBoostingRegressor (그레디언트 부스팅 회귀):
RandomForestRegressor (랜덤 포레스트 회귀):
AdaBoostRegressor (아다부스트 회귀):
ExtraTreesRegressor (엑스트라 트리 회귀):
LinearRegression (선형 회귀):